In [1]:
%cd ..

C:\Users\13661\PycharmProjects\KinshipVerification


In [2]:
import numpy as np
import torch
import torchvision
from torch.utils.data.dataloader import DataLoader

device = torch.device('cuda')

In [3]:
from dataset import Contrastive_pretrain

data_path = 'D:/文档/硕士/Thesis/UvA-NEMO_SMILE_DATABASE/aligned'


In [4]:
train_split = np.load('train.npy')
dataset = Contrastive_pretrain(data_path, train_split)
loader = DataLoader(
        dataset, batch_size=1, shuffle=False, num_workers=1)

In [5]:
from facenet_pytorch import InceptionResnetV1
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [6]:
from tqdm import tqdm
import torch.nn.functional as F

outs = []
with torch.no_grad():
    with tqdm(loader, unit="batch") as data_loader:
        for batch in data_loader:
            
            batch = [v.squeeze().permute(0,3,1,2).to(device) for v in batch]

            anchor = resnet(batch[0]).detach().mean(0).cpu()
            pos = resnet(batch[1]).detach().mean(0).cpu()
            neg = resnet(batch[2]).detach().mean(0).cpu()
            
            outs.append([F.normalize(torch.abs(anchor - pos), dim=0).numpy(), 1])
            outs.append([F.normalize(torch.abs(anchor - neg), dim=0).numpy(), 0])
#             outs.append([torch.cat([anchor, pos]).numpy(), 1])
#             outs.append([torch.cat([anchor, neg]).numpy(), 0])

100%|███████████████████████████████████████████████████████████████████████████| 1154/1154 [01:49<00:00, 10.54batch/s]


In [7]:
import random
random.shuffle(outs)

X = [a[0] for a in outs]
Y = [a[1] for a in outs]

In [8]:
print(len(X))

2308


In [9]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2, shuffle=True)

In [10]:
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score

clf = svm.SVC()

# scoring = {'prec_macro': 'precision_macro','rec_macro': make_scorer(recall_score, average='macro')}
# scores = cross_validate(clf,X, Y,
#                        scoring=scoring, cv=kf,
#                        return_estimator=True)
clf.fit(X,Y)
clf.score(X,Y)
# print(scores)

0.9566724436741768

In [11]:
# print(scores['test_prec_macro'])

In [13]:
test_split =  np.load('test.npy')
testset = Contrastive_pretrain(data_path, test_split)
testloader = DataLoader(
        testset, batch_size=1, shuffle=False, num_workers=1)

In [14]:
from tqdm import tqdm
import torch.nn.functional as F

t_outs = []
with torch.no_grad():
    with tqdm(testloader, unit="batch") as data_loader:
        for batch in data_loader:
            
            batch = [v.squeeze().permute(0,3,1,2).to(device) for v in batch]

            anchor = resnet(batch[0]).detach().mean(0).cpu()
            pos = resnet(batch[1]).detach().mean(0).cpu()
            neg = resnet(batch[2]).detach().mean(0).cpu()
            
            t_outs.append([F.normalize(torch.abs(anchor - pos), dim=0).numpy(), 1])
            t_outs.append([F.normalize(torch.abs(anchor - neg), dim=0).numpy(), 0])
#             t_outs.append([torch.cat([anchor, pos]).numpy(), 1])
#             t_outs.append([torch.cat([anchor, neg]).numpy(), 0])

100%|█████████████████████████████████████████████████████████████████████████████| 516/516 [00:48<00:00, 10.63batch/s]


In [15]:
random.shuffle(t_outs)

test_x = [a[0] for a in t_outs]
test_y = [a[1] for a in t_outs]

In [16]:
print(len(test_x))

1032


In [17]:
print(test_y)

[1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [18]:
clf.predict(test_x)

array([1, 0, 0, ..., 0, 0, 0])

In [19]:
clf.score(test_x, test_y)

0.5125968992248062